In [3]:
%matplotlib inline
import xarray as xr
import datetime
import os
import matplotlib.pyplot as plt
from dask.dot import dot_graph
import itertools
import logging
import netCDF4
import numpy as np
import numpy.ma as ma
import dask.array as da
from dask import delayed
import time
from dask.distributed import Client
from urllib import request
from multiprocessing import Pool
import glob

In [4]:
client = Client('scheduler:8786')
download_location = '/temp'
data_url = 'http://nasanex.s3.amazonaws.com'
# data_url = 'http://172.21.0.1:8080'
max_download_attempts = 5
all_models = ['ACCESS1-0',  'BNU-ESM', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM-CHEM', 'MIROC-ESM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M', 'bcc-csm1-1', 'inmcm4']
# all_models = ['ACCESS1-0',  'BNU-ESM']
all_vars = ['tasmax', 'pr', 'tasmin']
all_years = {
    'historical': list(range(1971, 2001))
}

year_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
year_leap_days = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# And some functions to deal with loading data into the cluster
def get_dataset_url(variable, scenario, model, year, prefix = data_url):
    prefix_filename = '/'.join([prefix, 'NEX-GDDP', 'BCSD', scenario, 'day', 'atmos', variable, 'r1i1p1', 'v1.0'])
    filename = '_'.join([variable, 'day', 'BCSD', scenario, 'r1i1p1', model, str(year) + '.nc'])
    url = '/'.join([prefix_filename, filename])
    # url = '/'.join([prefix, filename])
    return url
def get_context(year, **kwargs):
    variables = [kwargs.get('variable')] if kwargs.get('variable') else all_vars
    scenarios = ['historical']
    models = [kwargs.get('model')] if kwargs.get('model') else all_models
    outlist = []
    combinations = list(itertools.product(variables, scenarios, models))
    result = list(map(lambda comb: [ *comb, year ], combinations))
    return result

def get_year_ensemble(year, variable = 'tasmax'):
    context = get_context(year, variable = variable)
    datasets = list(map(lambda x: str(get_dataset_url(*x)), context))
    return datasets

def download_file(url):
    print("url: " + url)
    attempts = 0
    success = False
    filename = ""
    while attempts < max_download_attempts and not success:
        time.sleep(2 ** attempts)
        filename = '/'.join([download_location, str(url.split('/')[-1])])
        print("Downloading file at " + filename)
        u = request.urlopen(url)
        f = open(filename, 'wb')
        f.write(u.read())
        f.close()
        success = True
        break
    return filename

def destack(a, chunksize):
    nmodels, tasvars, time, lat, lon = a.shape
    nstacks_lat = int(np.ceil(lat / chunksize))
    nstacks_lon = int(np.ceil(lon / chunksize))
    
    stacks = []
    
    for i in range(nstacks_lat):
        for j in range(nstacks_lon):
            latmin, latmax = i * chunksize, (i+1) * chunksize
            lonmin, lonmax = j * chunksize, (j+1) * chunksize
            # print(i, j, '~>', latmin, latmax, lonmin, lonmax)
            stacked = a[:, :, :, latmin:latmax, lonmin:lonmax]
            # print(stacked)
            stacks.append(stacked)
    return stacks

def copy_dataset(src, output_filename):
    dst = netCDF4.Dataset('/home/jovyan/work/' + output_filename, "w")
    # copy attributes
    for name in src.ncattrs():
        dst.setncattr(name, src.getncattr(name))
    # copy dimensions
    for name, dimension in src.dimensions.items():
        dst.createDimension(
            name, (dimension)
        )
    dst.close()
    return output_filename

def restack(chunk_list, chunksize):
    shapes = list(map(np.shape, chunk_list))
    ndays = shapes[0][0]
    nlons = int(1440 / chunksize)
    nlats = int(720 / chunksize)
    out_array = np.empty((ndays, 720, 1440))
    combs = list(itertools.product(
        list(range(nlats)),
        list(range(nlons))
    ))
    res_list = zip(combs, chunk_list)
    for position, arr in res_list:
        minlon, maxlon = position[0] * chunksize, position[0] * chunksize + chunksize
        minlat, maxlat = position[1] * chunksize, position[1] * chunksize + chunksize
        out_array[:, minlon:maxlon, minlat:maxlat] = arr
    return out_array

def stack_from_disk(year, chunksize):
    year_ensemble_tasmin = list(map(lambda x: x.split('/')[-1], get_year_ensemble(year, variable = 'tasmin')))
    year_ensemble_tasmax = list(map(lambda x: x.split('/')[-1], get_year_ensemble(year, variable = 'tasmax')))
    datasets_tasmin = [ netCDF4.Dataset('../temp/' + filename) for filename in year_ensemble_tasmin ]
    datasets_tasmax = [ netCDF4.Dataset('../temp/' + filename) for filename in year_ensemble_tasmax ]
    das_tasmin = list(map(lambda dset: da.from_array(dset['tasmin'], chunks = (365, chunksize, chunksize)), datasets_tasmin))
    das_tasmax = list(map(lambda dset: da.from_array(dset['tasmax'], chunks = (365, chunksize, chunksize)), datasets_tasmax))
    # da.stack(list(map(lambda dset: da.from_array(dset['tasmax'], chunks = (365, 144, 144)), datasets_tasmax)))
    final_stack_tasmin = da.stack(das_tasmin)
    final_stack_tasmax = da.stack(das_tasmax)
    final_stack = da.stack((final_stack_tasmin, final_stack_tasmax))
    return final_stack

def days_to_ranges(years):
    ranges = [year_days[:i +1] for i, n in enumerate(year_days)]
    result = [(sum(element[:-1]), sum(element)) for element in ranges]
    return result

def stack_to_months(stack):
    days = days_to_ranges(year_days) if stack.shape[2] == 365 else days_to_ranges(year_leap_days)
    for period in days:
        yield(stack[:, :, period[0]:period[1], :, :])

def stack_to_models(stack):
    for model in range(stack.shape[0]):
        yield(stack[model, :, :, :])


In [10]:
def download_file_list(url_list):
    print("Starting download pool")
    pool = Pool()
    res = pool.map(download_file, url_list)
    print("Jobs sent")
    pool.close()
    pool.join()
    print("Downloads finished")
    print(res)
    return res

def download_and_stack(year, chunksize):
    tasmax_urls = list(map(lambda x: get_year_ensemble(x, variable = 'tasmax'), [year]))[0]
    print(tasmax_urls)
    tasmax_filenames = download_file_list(tasmax_urls)
    print(tasmax_filenames)
    tasmax_datasets = [ netCDF4.Dataset(filename) for filename in tasmax_filenames ]
    print(tasmax_datasets)

    tasmax_dask_arrays = []
    for dset in tasmax_datasets:
          tasmax_dask_arrays.append(da.from_array(dset['tasmax'], chunks= (366, 144, 144)))
    try:
          tasmax_final_stack = da.stack(tasmax_dask_arrays, axis = 0)
          print(tasmax_final_stack)
    except:
        return tasmax_filenames, None, None

    tasmin_urls = list(map(lambda x: get_year_ensemble(x, variable = 'tasmin'), [year]))[0]
    print(tasmin_urls)
    tasmin_filenames = download_file_list(tasmin_urls)
    print(tasmin_filenames)
    tasmin_datasets = [ netCDF4.Dataset(filename) for filename in tasmin_filenames ]
    print(tasmin_datasets)

    tasmin_dask_arrays = []
    for dset in tasmin_datasets:
          tasmin_dask_arrays.append(da.from_array(dset['tasmin'], chunks= (366, 144, 144)))
    try:
          tasmin_final_stack = da.stack(tasmin_dask_arrays, axis = 0)
          print(tasmin_final_stack)
    except:
          return [*tasmax_filenames, *tasmin_filenames], None, None

    final_stack = da.stack((tasmin_final_stack, tasmax_final_stack))
    return [*tasmax_filenames, *tasmin_filenames], final_stack



In [11]:
stack_year = download_and_stack(year, 144)
print(stack_year)

['http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1971.nc', 'http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_BNU-ESM_1971.nc', 'http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_CCSM4_1971.nc', 'http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_CESM1-BGC_1971.nc', 'http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_CNRM-CM5_1971.nc', 'http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_CSIRO-Mk3-6-0_1971.nc', 'http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_CanESM2_1971.nc', 'http://nasanex.s

url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_CNRM-CM5_1971.nc


url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_CCSM4_1971.nc


url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_BNU-ESM_1971.nc


url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1971.nc


...

url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_MIROC-ESM_1971.nc


url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_MIROC5_1971.nc


url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_MPI-ESM-LR_1971.nc


url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_MPI-ESM-MR_1971.nc


url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_MRI-CGCM3_1971.nc


In [6]:
year = 1971
stack_year = stack_from_disk(year, 144)
print(stack_year)
# dask.array<stack, shape=(2, 21, 365, 720, 1440), dtype=float32, chunksize=(1, 1, 365, 360, 360)>

dask.array<stack, shape=(2, 21, 365, 720, 1440), dtype=float32, chunksize=(1, 1, 365, 144, 144)>
